In [1]:
## Loading Libraries
import pandas as pd
import numpy as np
import sys
import jdatetime as jd
import os
import pickle

# import power_index_calculator as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
## Loading functions
from convert_ar_characters import convert_ar_characters
from ReadingBalancesheetData import read_blnc_data
from nthMax import nth_max
from gini import gini
from find_shapley import find_shapley
from find_banzhaf import find_banzhaf
from gameTheoric_concentration import gameTheoric_concentration
from fill_shapley_banzhaf import fill_shapley_banzhaf

# **Loading Shareholder Data**

In [3]:
## Loading DATA
os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
SDATA = pd.read_csv("Shareholder97.csv",index_col=0)

# Conver date from string to jdatetime
SDATA['True_Date'] = pd.to_datetime(SDATA['True_Date'], format='%Y-%m-%d')
G = SDATA.True_Date.drop_duplicates()
J = G.apply(lambda x: jd.date.fromgregorian(day=x.day,month=x.month,year=x.year))
DataOrg_date_GtoJ_dict = dict(zip(G,J))
SDATA['Jalali_Date']=SDATA.True_Date.map(DataOrg_date_GtoJ_dict)

SDATA.drop(columns=['High', 'Low', 'Open', 'Last', 'Volume', 'close',
       'True_Date', 'year', 'month', 'day', 'Fill_Flag','Unadjusted_close','chnk_id'],inplace=True)

In [4]:
# Creating Dataframe for saving concentration mearsurs
CMdf = SDATA.groupby('Symbol',as_index=False).agg({'Id_tse':'first','percent':'sum','ShareHolder':'count'}).rename(
    columns={'ShareHolder':'Num_holders','percent':'sum_over1'})
CMdf.reset_index(drop=True,inplace=True)

# **Concentration Measures**

## 1- Largest Owner

In [5]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':'max'}).rename(columns={'percent':'Largest_Owner'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left')

## 2- First/Second

In [6]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/nth_max(x,nth=2,interval=False)}}).rename(columns={'percent':'First_Second'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Second', '<lambda>'):'First_Second'})

C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\Mahdi\Anaconda3\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


## 3- First/Sumtwofour

In [7]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: max(x)/sum(nth_max(x,nth=[2,4],interval=True))}}).rename(
    columns={'percent':'First_Sumtwofour'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('First_Sumtwofour', '<lambda>'):'First_Sumtwofour'})

## 4- Sumfive

In [8]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum(nth_max(x,nth=[1,5],interval=True))}}).rename(columns={'percent':'Sumfive'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Sumfive', '<lambda>'):'Sumfive'})

## 5- Gini

We calculate gini coeficient using Deaton 1997:
$$ \gamma = \frac{N+1}{N-1} - \frac{2}{\mu\times N\times(N-1)}\sum_{i=1}^N{\rho_ix_i}$$

In [9]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: gini(list(x))}}).rename(columns={'percent':'Gini'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Gini', '<lambda>'):'Gini'})

## 6- Herfindhal

In [10]:
temp = SDATA.groupby('Symbol',as_index=False).agg({'percent':{lambda x: sum([(t/100)**2 for t in list(x)])}}).rename(columns={'percent':'Herfindhal'})
CMdf = pd.merge(CMdf,temp,left_on='Symbol',right_on='Symbol',how='left').rename(columns={('Herfindhal', '<lambda>'):'Herfindhal'})

## 7 & 8-Shapley-Shubik & Banzhaf

In [8]:
# # Initiating columns
# CMdf['SSCL'] = np.nan
# CMdf['SSCO'] = np.nan
# CMdf['SSDL'] = np.nan
# CMdf['SSDO'] = np.nan
# CMdf['BZCL'] = np.nan
# CMdf['BZCO'] = np.nan
# CMdf['BZDL'] = np.nan

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
CMdf = pd.read_csv('Measures.csv',index_col=0)

# CMdf = pd.merge(CMdf,CMdf_load[['Symbol','SSCL', 'SSCO', 'SSDL', 'SSDO', 'BZCL', 'BZCO', 'BZDL']],left_on='Symbol',right_on='Symbol',how='left')

In [12]:
%%time
data = fill_shapley_banzhaf(data = CMdf,SDATA=SDATA,fast_mode = True,time_pnt=True,major_thr = 5)
CMdf = data['CMdf']

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
CMdf.to_csv('Measures.csv')

print('len(Errors): ',len(data['Errors']))
data['Errors']
[x for x in data['Errors'] if x[2]!= 'Error: request error!']

The symbol  آریان  mode:  SSDL ,  1  from  10
The symbol  تکالا  mode:  SSDL ,  2  from  10
The symbol  سپرده  mode:  SSDL ,  3  from  10
The symbol  فرابورس  mode:  SSDL ,  4  from  10
The symbol  نبورس  mode:  SSDL ,  5  from  10
The symbol  نکالا  mode:  SSDL ,  6  from  10
The symbol  وتعاون  mode:  SSDL ,  7  from  10
The symbol  وپارس  mode:  SSDL ,  8  from  10
The symbol  وگردش  mode:  SSDL ,  9  from  10
The symbol  کیان  mode:  SSDL ,  10  from  10


**About  0  out of  10  of mising cells are filled!
len(Errors):  10
Wall time: 10min 57s


[]

# **Loading Balancesheet Data and Merging**

In [13]:
blnc_data = read_blnc_data(file='98.txt')
blnc_data['book_value'] = blnc_data.Tot_asset-blnc_data.Tot_lib
blnc_data = blnc_data[blnc_data.book_value>0]

In [15]:
data_out = pd.merge(CMdf,blnc_data[['Symbol','Tot_asset', 'Tot_lib', 'Capital', 'Equity_at_year_end','Debt_Equity', 'Debt_Equity_normal', 'book_value']]
                    ,left_on='Symbol',right_on='Symbol',how='outer')

os.chdir(r"C:\Users\Mahdi\OneDrive\Master Thesis\Data")
data_out.to_csv('data_owenership97_blnc98.csv')

In [10]:
CMdf[CMdf.Num_holders<5]

,Symbol,Id_tse,sum_over1,Num_holders,Largest_Owner,First_Second,First_Sumtwofour,Sumfive,Gini,Herfindhal,SSCL,SSCO,SSDL,SSDO,BZCL,BZCO,BZDL
0,اعتضاد غدیر,34973883374080119,99.89,1,99.89,1.000000,1.000000,99.89,0.000000,0.997801,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.0
1,کاغذ مراغه,21383339313241074,51.53,1,51.53,1.000000,1.000000,51.53,0.000000,0.265534,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.0
3,آتیمس,22839330962768817,98.98,1,98.98,1.000000,1.000000,98.98,0.000000,0.979704,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.0
6,آساس,66682662312253625,99.23,1,99.23,1.000000,1.000000,99.23,0.000000,0.984659,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.0
7,آسام,36592972482259020,99.47,1,99.47,1.000000,1.000000,99.47,0.000000,0.989428,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,گنگین,59470107928175959,99.25,2,79.25,3.962500,3.962500,99.25,0.596977,0.668056,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.0
601,گوهران,65018804181564924,94.14,1,94.14,1.000000,1.000000,94.14,0.000000,0.886234,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.0
602,گپارس,59848307608894801,30.07,4,20.90,3.181126,2.279171,30.07,0.716439,0.048351,0.259237,0.652238,0.261908,0.649208,0.519212,0.439733,1.0
603,گکوثر,66599109405217136,87.82,4,46.00,1.380138,1.099952,87.82,0.586275,0.326294,0.610347,0.179688,0.601808,0.198864,0.792582,0.137922,1.0
